In [ ]:
import numpy as np
import g3utils as ut
import matplotlib.pyplot as plt
import so3g.proj
from spt3g import core
import pathlib

In [ ]:
control_computer_g3_dir = pathlib.Path("/media/player1/blast2020fc1/blasttng_g3")
roach1_pass3_file = control_computer_g3_dir / "testing/roach1_pass3.g3"

ra_df_added = control_computer_g3_dir / "mapmaking/ra_df_added.g3"
norm_df_added = control_computer_g3_dir / "mapmaking/norm_df_added.g3"

In [ ]:
stats = ut.DetectorStats(data_key="df")

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(ra_df_added))
pipe.Add(stats)
pipe.Run()

detector_medians = np.median(np.array(stats.medians), axis=0)
detector_medians.shape
plt.hist(detector_medians, bins=50)
plt.xticks(rotation=40)
plt.xlabel("Detector Median")
plt.ylabel("Number of Detectors")
plt.show()

detector_stds = np.median(np.array(stats.stds), axis=0)
detector_stds.shape
plt.hist(detector_stds, bins=50)
plt.xticks(rotation=40)
plt.xlabel("Detector Standard Deviation")
plt.ylabel("Number of Detectors")
plt.gca().set_yscale("log")
plt.show()

In [ ]:
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(roach1_pass3_file))
pipe.Add(ut.AddAllDF)
pipe.Add(ut.add_cal_lamp_df, iq_key="cal_lamp_data")
pipe.Add(ut.NormalizeDF, detector_medians=detector_medians)
pipe.Add(ut.FrameCounter)
pipe.Add(core.G3Writer, filename=str(norm_df_added))
pipe.Run()

In [ ]:
def first_frame(file, type=core.G3FrameType.Scan):
    grabber = ut.FirstFrameGrabber(frame_type=type)
    pipe = core.G3Pipeline()
    pipe.Add(core.G3Reader, filename=file)
    pipe.Add(grabber)
    pipe.Run()
    return grabber.first_frame

kids = np.array(ut.first_scan_frame(str(norm_df_added))["norm_df"].names)

In [ ]:
# center of the sky map
ra0 = 231.15 * core.G3Units.deg
dec0 = -56.2 * core.G3Units.deg

# map dimensions
xlen = 1.4 * core.G3Units.deg
ylen = 0.9 * core.G3Units.deg

# pixel resolution
res = 1 * core.G3Units.arcmin

source_coords = {}
# determine source shifts
single_kid_binners = []
for kid in kids:
    binner = ut.SingleMapBinner(
        kid,
        timestreams="df",
        ra0=ra0,
        dec0=dec0,
        xlen=xlen,
        ylen=ylen,
        res=res
    )
    pipe = core.G3Pipeline()
    pipe.Add(core.G3Reader, filename=str(norm_df_added))
    pipe.Add(binner)
    pipe.Run()
    source_coords[kid] = binner.source_coords()
    single_kid_binners.append(binner)
    print(f"source_coords for {kid}: {source_coords[kid]}")

# for binner in single_kid_binners:
#     binner.plot()

In [ ]:
xs=[]
ys=[]
for key in source_coords.keys():
    xs.append(source_coords[key][0])
    ys.append(source_coords[key][1])
plt.scatter(xs, ys)

center_kids = [key for key in source_coords.keys() if (30 <= source_coords[key][0] <= 65) and (15 <= source_coords[key][1] <= 30)]

xs=[]
ys=[]
for key in center_kids:
    xs.append(source_coords[key][0])
    ys.append(source_coords[key][1])
plt.scatter(xs, ys)
plt.show()

In [ ]:
rcw_92_min_lat = -77.110
rcw_92_max_lat = -77.070
rcw_92_min_lon = 162.20
rcw_92_max_lon = 162.55
rcw_92_min_alt = 36030
rcw_92_max_alt = 36120
rcw_92_avg_lat = rcw_92_min_lat / 2. + rcw_92_max_lat / 2.
rcw_92_avg_lon = rcw_92_min_lon / 2. + rcw_92_max_lon / 2.
rcw_92_avg_alt = rcw_92_min_alt / 2. + rcw_92_max_alt / 2.
BLASTTNG_SITE = so3g.proj.EarthlySite(rcw_92_avg_lon, rcw_92_avg_lat, rcw_92_avg_alt)  # we could also add weather

In [ ]:
binner = ut.MapBinner(
    timestreams="df",
    source_coords=source_coords,
    ra0=ra0, dec0=dec0, xlen=xlen, ylen=ylen, res=res,
    select_kids=center_kids
)

# create the pipeline
pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(norm_df_added))
pipe.Add(binner)
pipe.Add(ut.FrameCounter)
pipe.Run()

binner.plot()

In [ ]:
grabber = ut.FirstFrameGrabber(frame_type=core.G3FrameType.Calibration)

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(roach1_pass3_file))
pipe.Add(ut.add_cal_lamp_df, roach_id=1, iq_key="cal_lamp_data")
pipe.Add(grabber)
pipe.Run()

cal_frame = grabber.first_frame

print(cal_frame)